In [11]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm

In [12]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

In [13]:
def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

In [23]:
# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.05)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=1.0) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.variational.MCState(self.sampler, self.machine, n_samples=1000, n_discard=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output):
        self.vs.init_parameters(nk.nn.initializers.normal(stddev=0.25))
        gs = nk.VMC(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, sr=self.sr)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=1000)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, runTime

In [24]:
# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr


# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/Logs"+str(N)) #where _call_ will be invoked
    return eng, state, runTime

In [25]:
#For N=5 only
N = 5
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
#A = N/2
#N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization(ha)
#Ground state energy
edEng = e[0]
# Ground state
edState = v[0]

#Lists for Histogram Data
numRuns = 50
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []

# Get errors for each run in histogram
for i in range(len(hisIt)):
    engTemp, stateTemp, runTimeTemp = runDescentCS(N,B,Ak,alpha)
    #engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)

#Save data to JSON file
data = [engErr, stateErr, runTime]
fileName = "2021_summer_data/N"+str(N)+"M" + str(M)+"_1.0_systematic_con.json"
open(fileName, "w").close()
with open(fileName, 'a') as file:
    for item in data:
        line = json.dumps(item)
        file.write(line + '\n')
print('SAVED')

  1%|          | 11/1000 [00:00<00:09, 105.32it/s, Energy=0.240-0.013j ± 0.030 [σ²=0.923, R̂=1.0041]]

Eng error  [0.007033821866924361]
State error  [0.050085159437594284]


  1%|          | 11/1000 [00:00<00:10, 98.52it/s, Energy=0.190-0.042j ± 0.037 [σ²=1.369, R̂=1.0066]]

Eng error  [0.007033821866924361, 0.011875537048148255]
State error  [0.050085159437594284, 0.04022434574775491]


  1%|          | 11/1000 [00:00<00:09, 103.27it/s, Energy=0.736+0.006j ± 0.021 [σ²=0.451, R̂=1.0093]]

...

Eng error  [0.007033821866924361, 0.011875537048148255, 0.011574304817997039, 0.0032472583735148053, 0.00738456558508549, 0.35635873809788743, 0.00754876231302104, 0.010944819978023546, 0.016846631086851804, 0.008518183381389433, 0.008200526053592005, 0.009028343040316189, 0.006583509070176463, 0.029522605359422638, 0.013843237895369453, 0.0019522074109326049, 0.008256180344001107, 0.001848076395426279, 0.025384330515239473, 0.34798712200813764, 0.3574588846518614, 0.10250922284490405, 0.008230591670603804, 0.010892392813853569, 0.0023138187227984552, 0.0039620585185984325, 0.013111757954027414, 0.009924350046844665, 0.00552020785692986, 0.004003011254149369, 0.004616643362541639, 0.002717360502134003, 0.004951292372549654, 0.009017959965404554, 0.02435921431983612, 0.01209742015606996, 0.005009081315298758, 0.003705525836267709, 0.003961318814772685, 0.028942392939220518, 0.00952726612839383, 0.003494291616051859, 0.008123654279756476, 0.007703055763582878, 0.0046016762511016385, 0.01

  1%|          | 9/1000 [00:00<00:12, 81.12it/s, Energy=0.557-0.014j ± 0.031 [σ²=0.976, R̂=1.0032]]

Eng error  [0.007033821866924361, 0.011875537048148255, 0.011574304817997039, 0.0032472583735148053, 0.00738456558508549, 0.35635873809788743, 0.00754876231302104, 0.010944819978023546, 0.016846631086851804, 0.008518183381389433, 0.008200526053592005, 0.009028343040316189, 0.006583509070176463, 0.029522605359422638, 0.013843237895369453, 0.0019522074109326049, 0.008256180344001107, 0.001848076395426279, 0.025384330515239473, 0.34798712200813764, 0.3574588846518614, 0.10250922284490405, 0.008230591670603804, 0.010892392813853569, 0.0023138187227984552, 0.0039620585185984325, 0.013111757954027414, 0.009924350046844665, 0.00552020785692986, 0.004003011254149369, 0.004616643362541639, 0.002717360502134003, 0.004951292372549654, 0.009017959965404554, 0.02435921431983612, 0.01209742015606996, 0.005009081315298758, 0.003705525836267709, 0.003961318814772685, 0.028942392939220518, 0.00952726612839383, 0.003494291616051859, 0.008123654279756476, 0.007703055763582878, 0.0046016762511016385, 0.01

100%|██████████| 1000/1000 [00:16<00:00, 59.69it/s, Energy=-1.8188-0.0006j ± 0.0038 [σ²=0.0143, R̂=0.9994]]


Eng error  [0.007033821866924361, 0.011875537048148255, 0.011574304817997039, 0.0032472583735148053, 0.00738456558508549, 0.35635873809788743, 0.00754876231302104, 0.010944819978023546, 0.016846631086851804, 0.008518183381389433, 0.008200526053592005, 0.009028343040316189, 0.006583509070176463, 0.029522605359422638, 0.013843237895369453, 0.0019522074109326049, 0.008256180344001107, 0.001848076395426279, 0.025384330515239473, 0.34798712200813764, 0.3574588846518614, 0.10250922284490405, 0.008230591670603804, 0.010892392813853569, 0.0023138187227984552, 0.0039620585185984325, 0.013111757954027414, 0.009924350046844665, 0.00552020785692986, 0.004003011254149369, 0.004616643362541639, 0.002717360502134003, 0.004951292372549654, 0.009017959965404554, 0.02435921431983612, 0.01209742015606996, 0.005009081315298758, 0.003705525836267709, 0.003961318814772685, 0.028942392939220518, 0.00952726612839383, 0.003494291616051859, 0.008123654279756476, 0.007703055763582878, 0.0046016762511016385, 0.01